In [54]:
import re
import numpy as np


# Задание

Смоделировать игру против лотерейного автомата типа “777”. Игрок платит 1 руб., после чего выпадает случайное целое число, равномерно распределенное от 0 до 999. При некоторых значениях числа игрок получает выигрыш (см. справа)

Выгодна ли игра игроку?

Сколько в среднем приобретает
или теряет игрок за одну игру?

777: 200 руб.
999: 100 руб.
555: 50 руб.
333: 15 руб.
111: 10 руб.
*77: 5 руб.
**7: 3 руб.
*00: 2 руб.
**0: 1 руб.

– любая цифра

**допущение применительно к постановке задачи**:
- под "целым числом" подразумевается комбинация трех цифр. Примеры: 001, 050, 501


In [137]:
# Словарь с выигрышными комбинациями и суммами выигрышей. Используется нотация записи регулярных выражений
dict_win_figures_and_values = {
    '777':200,
    '999':100,
    '555':50,
    '333':15,
    '111':10,
    '.77':5,
    '..7':3,
    '.00':2,
    '..0':1,    
}
# dict_win_figures_and_values

## Теоретический расчет среднего выигрыша (мат.ожидание)

In [120]:
no_of_win_figures = 0
sum_of_values = 0

# перебор всех комбинаций от 000 до 999
for figure in range(1000):
# конвертация int к 3-значной комбинации (примитивный способ)
    if figure < 10:
        figure_3digit = '00' + str(figure)
    elif figure < 100:
        figure_3digit = '0' + str(figure)
    else:
        figure_3digit = str(figure)

# подбор подходящей маски для комбинации, если подошла - определение ее выигрыша
    for regex_win_figure in list(dict_win_figures_and_values.keys()):
        if re.match( regex_win_figure, figure_3digit ):
# мат.ожидание выигрыша: сумма произведений вероятности выпадения каждой комбинации (1/1000) на ее выигрыш                
            no_of_win_figures += 1
            sum_of_values += 1/1000 * dict_win_figures_and_values[regex_win_figure]
#             print( figure_3digit, regex_win_figure, dict_win_figures_and_values[regex_win_figure] )
# выход из цикла, потому что одна комбинация приносит только один выигрыш
            break

print( f'Мат.ожидание выигрыша на вложенный рубль: {sum_of_values} рублей, доля выигрышных комбинаций: {no_of_win_figures / 1000}' )

Мат.ожидание выигрыша на вложенный рубль: 0.8000000000000004 рублей, доля выигрышных комбинаций: 0.204


## Моделирование игры
Используется генератор случайных чисел и фиксированное количество попыток. С увеличением количества попыток результат должен приближаться к теоретическому.

In [136]:
trials = 100000    # Количество попыток

no_of_win_trials = 0
sum_of_win_trials = 0

for trial in range(trials):
# генерация рандомного int от 0 до 999 (1000 указано потому, что правая граница не включается) и приведение к 3-значной комбинации
    random_int = np.random.randint(0, 1000)    
    if random_int < 10:
        random_3digit = '00' + str(random_int)
    elif random_int < 100:
        random_3digit = '0' + str(random_int)
    else:
        random_3digit = str(random_int)

    is_win_trial = 0
    for regex_win_figure in list(dict_win_figures_and_values.keys()):
        if re.match( regex_win_figure, str(random_3digit) ):
            is_win_trial = 1
            no_of_win_trials += 1
            sum_of_win_trials += dict_win_figures_and_values[regex_win_figure]
            break
    
#     if is_win_trial:
#         print(random_int, random_3digit, regex_win_figure, dict_win_figures_and_values[regex_win_figure])
#     else:
#         print(random_int, random_3digit)

print( f'Количество попыток: {trials}, доля выигрышных: {no_of_win_trials / trials}, выигрыш на вложенный рубль: {sum_of_win_trials / trials} рублей' )

Количество попыток: 100000, доля выигрышных: 0.20373, выигрыш на вложенный рубль: 0.80112 рублей


## Выводы 
Разумеется, в среднем на большом количестве попыток **игрок теряет**, а не приобретает. Примерный размер потерь - 20 копеек на потраченный рубль (т.е. на одну игру при цене 1р). 